In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

In [ ]:
rate_to_use = ['stre', 'spd', 'jmp', 'endu', 'ins', 'dnk', 'ft', 'fg', 'tp', 'oiq', 'diq', 'drb', 'pss', 'reb', 'hgt']

In [ ]:
sim = json.load(open('newresign.json','rb'))
data = json.load(open('real-player.json','rb'))

In [ ]:
picks = defaultdict(list)
X = []
y = []
i =0 
for p in sim['players']:
    if int(p['draft']['year']) < 2021:
        continue
    pick = (p['draft']['round']-1)*30 + p['draft']['pick']
    vec = [p['ratings'][0][k] for k in rate_to_use]
    vec = np.array(vec)
    X.append(vec)
    y.append(p['ratings'][0]['ovr'])
    picks[pick].append(vec)
    i+=1

In [ ]:
import statsmodels.api as sm
# good enough
clf = sm.OLS(y,sm.add_constant(X)).fit()

In [ ]:
picks_a = {k: np.array(v) for k,v in picks.items()}
picks_o = {k: np.mean([clf.predict([1] + list(p)) for p in v]) for k,v in picks.items()}
picks_v = {k: np.std([clf.predict([1] + list(p)) for p in v]) for k,v in picks.items()}

In [ ]:
picks_norm = {k: np.array(v) for k,v in picks.items() if len(v) > 1}
picks_norm = {k: v/v.sum(1,keepdims=True) for k,v in picks_norm.items()}

In [ ]:
roster = json.load(open('1947_preseason.json','rb'))

In [ ]:
for p in roster['players']:
    if 'srID' not in p:
        continue
    srId = p['srID']
    if srId not in data['bios']:
        continue
    bio = data['bios'][srId]
    pick = (bio['draftRound']-1)*30 + bio['draftPick']
    vec = np.array([p['ratings'][0][k] for k in rate_to_use])
    vecN = vec/vec.sum()
    if pick not in picks_norm:
        pick = -30
    #diff = np.linalg.norm(picks_norm[pick]- vecN,axis=1)
    #diff_idx = np.argsort(diff)[:3]
    #stats = np.mean(picks_a[pick][diff_idx],0)
    target_ovr = picks_o[pick] + np.random.randn()*picks_v[pick]
    n = (target_ovr - clf.params[0])/np.sum(clf.params[1:]*vecN)
    stats =vecN*n
    
    del p['ratings'][0]['ovr']
    del p['ratings'][0]['pot']
    del p['ratings'][0]['fuzz']
    del p['ratings'][0]['skills']
    del p['ratings'][0]['pos']

    for k,v in zip(rate_to_use,stats):
        p['ratings'][0][k] = int(round(np.clip(v,0,100)))
    #if srId == 'alexacl01':
    #    print(pick,np.mean(stats),diff_idx)

In [ ]:
with open('1947_var.json','wt') as fp:
    json.dump(roster,fp)